In [1]:
import os
import sys
import pandas as pd
import csv
import json
import sqlite3
from bokeh.charts import *

## Parameters you can change

# Abs path to settings file
root = "C:/DS/Github/MusicRecommendation"  # BA, Windows

# Select the dataset to view
datasetToUse ="inputfile_sml" # inputfile , inputfile_sml , or inputfile_verysml


## Finish setting up
os.chdir(root)
## Add the prelim module
fPath = root + "/1_codebase"
if fPath not in sys.path: sys.path.append(fPath)
from codebase import *

settingsDict =  loadSettings()

<a id='Load Data'></a>
<h3 style="background-color:#616161;color:white">Load Data Into SQLite3</h3>

In [2]:
inputFilePath = root + settingsDict[datasetToUse]
dbPath = root + settingsDict['dbPath']

db = sqlite3.connect(dbPath)

colHeadings=['UserID','PlayedTimestamp','ArtistID','ArtistName','TrackID','TrackName']
dataFormat={'UserID': str, 'PlayedTimestamp': str, 'ArtistID' :str, 'ArtistName': str, 'TrackID': str, 'TrackName': str}
parse_dates = ['PlayedTimestamp']

inpData = pd.read_csv(inputFilePath, sep='\t', error_bad_lines= False,quoting=csv.QUOTE_NONE, header=None,names=colHeadings, dtype=dataFormat,parse_dates=parse_dates)
inpData.to_sql('tblInputData', db, flavor='sqlite',
                                            schema=None, if_exists='replace', index=True,
                                            index_label=None, chunksize=None, dtype=None)

db.close()